In [7]:
from elasticsearch import Elasticsearch
from flask import Flask, request, jsonify
import redis
import csv
import msgpack
import pandas as pd
import time
import argparse
import difflib
import json
from collections import defaultdict
import time
es = Elasticsearch(['http://localhost:9200'])


In [8]:
class TrieNode_dict:
    def __init__(self, letter = None):
        self.children = defaultdict(TrieNode_dict)
        self.is_word = False
        self.description = None
        self.letter = letter

class Trie_dict:
    def __init__(self):
        self.root = TrieNode_dict()
        self.count = 0
        
    def insert(self, word, description=None):
        current = self.root
        for char in word:
            current = current.children[char]
        if not current.is_word:
            current.is_word = True
            self.count += 1
        current.description = description
    
    def search(self, word):
        current = self.root
        for char in word:
            if char not in current.children:
                return None
            current = current.children[char]
        if current.is_word:
            return current.description
        return None

    def insert_list(self, lst):
        for word in lst:
            self.insert(word) 

    def size(self):
        return self.count
    
    def insert_dict(self, dict_obj):
        for key, definition in dict_obj.items():
            self.insert(key, definition)
    
    def fuzzy_search(self, word, cutoff=0.6):
        results = difflib.get_close_matches(word, self.words(), n=10, cutoff=cutoff)
        return {result: (self.search(result), difflib.SequenceMatcher(None, word, result).ratio()) for result in results}
        
    def words(self):
        words = []
        def dfs(node, word):
            if node.is_word:
                words.append(word)
            for char in node.children:
                dfs(node.children[char], word + char)
        dfs(self.root, "")
        return words
    

In [9]:
import pickle
trie_dict_test = Trie_dict()
with open('english_dict.pkl', 'rb') as f:
    loaded_dict = pickle.load(f)
trie_dict_test.insert_dict(loaded_dict)

In [12]:
def insert_trie_in_elasticsearch(trie, index_name, es):
    if es.indices.exists(index=index_name):
        es.indices.delete(index=index_name)

    es.indices.create(index=index_name)

    def add_node_to_index(node, prefix=''):
        doc = {
            'is_word': node.is_word,
            'description': node.description
        }

        for char, child_node in node.children.items():
            child_prefix = prefix + char
            add_node_to_index(child_node, child_prefix)

        es.index(index=index_name, body=doc, id=prefix)

    add_node_to_index(trie.root)


In [14]:
insert_trie_in_elasticsearch(trie_dict_test, 'root', es)

/home/gmartini2019/miniconda3/envs/bruce_environment/lib/python3.7/site-packages/ipykernel_launcher.py:2: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.17/security-minimal-setup.html to enable security.
  
/home/gmartini2019/miniconda3/envs/bruce_environment/lib/python3.7/site-packages/ipykernel_launcher.py:5: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.17/security-minimal-setup.html to enable security.
  """
/home/gmartini2019/miniconda3/envs/bruce_environment/lib/python3.7/site-packages/ipykernel_launcher.py:17: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use the 'document' parameter. See https://gith